### Preparing the dataset

Prepare the dataset using Labelme annotation tool (for Instance segmentation) and LabelImg for object detection.


### Installing Detectron2

In [1]:
# install dependencies: 
#!pip install pyyaml==5.1
#!pip install onnx==1.8.0

#!pip install pyyaml==5.1
#!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

!gcc --version

gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



### Export Model to ONNX

In [3]:
import cv2
import detectron2
import onnx
import torch
import numpy as np

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.export import export_onnx_model, export_caffe2_model
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
import detectron2.data.transforms as T

# From Module:
from mask_inference import load_maskrcnn

In [4]:
def onnx_export(model_path, classes_list, conf_thresh, img_size, save_name):
    
    '''
        Method for converting .pt detectron2 mask-rcnn model to onnx
    '''
    
    print('Loading Model...')
    cfg, _, _ = load_maskrcnn(model_path, classes_list, conf_thresh)

    model = build_model(cfg)
    model.eval()
    checkpointer = DetectionCheckpointer(model)
    checkpointer.load(cfg.MODEL.WEIGHTS)
    aug = T.ResizeShortestEdge([cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST],
                               cfg.INPUT.MAX_SIZE_TEST)

    print('Min Input:', cfg.INPUT.MIN_SIZE_TEST)
    print('Max Input:', cfg.INPUT.MAX_SIZE_TEST)
    
    height, width, channels = img_size
    
    print('Collecting valid input img:')
    !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
    img = cv2.imread("./input.jpg")
    
    image = aug.get_transform(img).apply_image(img)
    image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
    inputs = {"image": image, "height": height, "width": width}
    
    # Export to Onnx model
    print('Exporting model to onnx...')
    onnxModel = export_onnx_model(cfg, model, [inputs])
    onnx.save(onnxModel, f'{save_name}.onnx')
    print('Process completed, model onnx available!')

In [6]:
model_path   = '/home/autaza/Documentos/detectron_2_venv/fringe_segmentation.pth'
classes_list = ['fringe']
conf_thresh  = 0.5
img_size     = (2000, 2000, 3)
save_name    = 'fringe_seg'

onnx_export(model_path, classes_list, conf_thresh, img_size, save_name)

Loading Model...
Min Input: 800
Max Input: 1333


export_caffe2_model() is deprecated. Please use `Caffe2Tracer().export_onnx() instead.


Exporting model to onnx...
All further enhancements and fixes to optimizers will be done in this new repo.
The optimizer code in onnx/onnx repo will be removed in 1.9 release.

Process completed, model onnx available!
